Step 1: Install and Import Required Libraries

In [1]:
!pip install --quiet google-cloud-bigquery

In [3]:
from google.cloud import bigquery

Step 2: Set Project and Create Dataset

In [4]:
PROJECT_ID = "qwiklabs-gcp-01-79d56652b122"
DATASET_ID = "weather_gemini_lab"

client = bigquery.Client(project=PROJECT_ID)

dataset_ref = f"{PROJECT_ID}.{DATASET_ID}"
dataset = bigquery.Dataset(dataset_ref)
dataset.location = "US"

client.create_dataset(dataset, exists_ok=True)

print(f"Dataset `{dataset_ref}` is ready.")


Dataset `qwiklabs-gcp-01-79d56652b122.weather_gemini_lab` is ready.


Step 3: Import Weather Data into BigQuery

In [5]:
raw_table_id = f"{PROJECT_ID}.{DATASET_ID}.weather_data"

job_config = bigquery.LoadJobConfig(
    source_format=bigquery.SourceFormat.CSV,
    skip_leading_rows=1,
    autodetect=True,
)

source_uri = "gs://labs.roitraining.com/data-to-ai-workshop/weather_data.csv"

client.load_table_from_uri(
    source_uri,
    raw_table_id,
    job_config=job_config
).result()

print("Weather data loaded into BigQuery.")


Weather data loaded into BigQuery.


Step 4: Study the Weather Data

In [7]:
client.query(f"""
select *
from `{raw_table_id}`
limit 20
""").to_dataframe()


,date,city,state,temperature_f,wind_speed_mph,precipitation_in,barometric_pressure_inHg,humidity_percent,weather_condition
0,2025-02-21,Atlanta,GA,55.7,5.0,0.12,29.80,50.4,Cloudy
1,2025-02-26,Atlanta,GA,75.2,10.4,0.03,29.58,49.9,Cloudy
2,2025-03-01,Atlanta,GA,51.7,4.7,0.08,29.74,49.9,Cloudy
3,2025-03-05,Atlanta,GA,74.4,5.1,0.02,29.92,50.4,Cloudy
4,2025-03-10,Atlanta,GA,59.5,9.6,0.09,29.67,57.2,Cloudy
5,2025-03-14,Atlanta,GA,71.7,7.2,0.18,29.92,55.3,Cloudy
6,2025-02-19,Boston,MA,61.7,3.9,0.11,29.62,54.1,Cloudy
7,2025-03-09,Boston,MA,76.7,4.3,0.09,29.52,40.9,Cloudy
8,2025-03-13,Boston,MA,71.9,9.8,0.16,29.99,42.3,Cloudy
9,2025-03-19,Boston,MA,60.7,6.4,0.04,29.83,49.4,Cloudy


In [8]:
client.query(f"""
select
  column_name,
  data_type
from `{PROJECT_ID}.{DATASET_ID}.INFORMATION_SCHEMA.COLUMNS`
where table_name = 'weather_data'
order by ordinal_position
""").to_dataframe()


,column_name,data_type
0,date,DATE
1,city,STRING
2,state,STRING
3,temperature_f,FLOAT64
4,wind_speed_mph,FLOAT64
5,precipitation_in,FLOAT64
6,barometric_pressure_inHg,FLOAT64
7,humidity_percent,FLOAT64
8,weather_condition,STRING


Step 5: Create Gemini Model in BigQuery

In [14]:
MODEL_NAME = "gemini_weather_model"

query = f"""
create or replace model `{PROJECT_ID}.{DATASET_ID}.{MODEL_NAME}`
remote with connection default
options (
  endpoint = 'gemini-2.0-flash-001'
);
"""

client.query(query).result()
print("Gemini REMOTE model created successfully.")


Gemini REMOTE model created successfully.


Step 6: Generate Weather Reports with ML.GENERATE_TEXT

In [15]:
report_table_id = f"{PROJECT_ID}.{DATASET_ID}.weather_with_report"

client.query(f"""
create or replace table `{report_table_id}` as
select
  ml_generate_text_result['candidates'][0]['content']['parts'][0]['text']
    as weather_report,
  *
from ML.GENERATE_TEXT(
  model `{PROJECT_ID}.{DATASET_ID}.{MODEL_NAME}`,
  (
    select
      concat(
        'Generate a concise weather report or warning based on the following weather data: ',
        TO_JSON_STRING(t)
      ) as prompt,
      *
    from `{raw_table_id}` t
  ),
  STRUCT(
    0.2 AS temperature,
    512 AS max_output_tokens
  )
);
""").result()

print("Weather reports generated.")


Weather reports generated.


Step 7: Review Generated Reports

In [16]:
client.query(f"""
SELECT
  weather_report
FROM `{report_table_id}`
LIMIT 10
""").to_dataframe()


,weather_report
0,"""Boston, MA: Sunny and warm today, March 1st, ..."
1,"""Atlanta, GA: Sunny and hot today, March 18th,..."
2,"""Atlanta, GA: Sunny and mild today, Feb 28th. ..."
3,"""Chicago, IL: Sunny and mild today, March 2nd,..."
4,"""Seattle, WA: Light snow today, March 18th. Te..."
5,"""Boston, MA: Sunny and warm today, March 14th,..."
6,"""**Boston, MA - March 12, 2025: Heavy snow. Te..."
7,"""Houston, TX: Cloudy with light rain today, Fe..."
8,"""Houston, TX: Sunny and warm today, March 15th..."
9,"""Houston, TX: Sunny and mild today, March 19th..."
